In [ ]:
import numpy as np
import pandas as pd
import time
from scipy import ndimage
from PIL import Image
from matplotlib import pyplot as plt

## Simple and fast game creation, keeping only the first gen and the last as we can see on train dataset.


#### Rules for creation https://www.kaggle.com/c/conways-reverse-game-of-life-2020/data


> * **Rules**

1.  An initial board was chosen by filling the board with a random density between 1% full (mostly zeros) and 99% full (mostly ones).
1.  This initial board was evolved 5 steps.
1.  The starting board's state was recorded after the 5 "warmup steps". These are the values in the start variables.
1.  The starting board was then evolved delta steps. Delta was chosen to be uniformly random between 1 and 5. If the stopping board was empty, the game was discarded.
1.  The stopping board's state was then recorded. These are the values in the stop variables.

## Classes

In [ ]:
# Class responsible for generating n games.

class gameFactory:
    def __init__(self,howmany):
        self.howmany = howmany # how many games we want to have
        value = howmany//5 #5 bins
        self.distri = np.array([1]*value+[2]*value+[3]*value+[4]*value+[5]*value)
        self.games = []
        
    def generate(self,size=25):
        for k in self.distri:
            actual_games = []
            actual = gameCreator(size)
            created = False
            while not(created):
                for i in range(k):
                    actual.nextGen()
                if actual.gens[-1].sum()>0: created=True
                else: actual = gameCreator(size)
            
            actual_games = [actual.gens[0],actual.gens[-1]]
            self.games.append(actual_games)
    
    # Converts created games into a dataframe
    def to_df(self):
        d = {'delta': self.distri}
        df_delta = pd.DataFrame(data=d)
        
        df_columns = pd.read_csv("../input/conways-reverse-game-of-life-2020/train.csv").drop(["id","delta"],axis=1).columns
                                 
        ls_ = []
        for k in self.games:
            ls = np.concatenate((k[0].flatten() , k[1].flatten()), axis=0)
            ls_.append(list(ls))
     
        df_data = pd.DataFrame(data=ls_, 
                                columns = df_columns)
        
        return pd.concat([df_delta, df_data], axis=1, sort=False)
                

In [ ]:
# Class responsible for creating a single game.

class gameCreator:
    def __init__(self, grid):
        self.size = grid
        created = False
        while not(created):
            self.grid = np.random.randint(2, size=(grid,grid))
            distribution = self.grid.sum()/(grid*grid)
            if distribution > 0 or distribution < 1:
                created = True
        self.kernel = [[1,1,1],[1,0,1],[1,1,1]]
        self.gens = [self.grid]
        self.currentGen = 0
        for k in range(5):
            self.nextGen() # warmups
            self.currentGen = 0
            
        self.gens = self.gens[5:]
        #warmup setup ok
    
    def rules(self,number,alive):
        if not(alive): 
            if number==3: return 1
            else: return 0
        if alive:
            if number==1: return 0
            if number==2: return 1
            if number==3: return 1
            if number>=4: return 0
            else: return 0

    
    def filterApply(self,gen):
        return (ndimage.convolve(np.pad(self.gens[gen],
                                    [(1, ), (1, )], 
                                    mode='constant'),
                                     self.kernel))[1:-1,1:-1]
         
    
    def nextGen(self):
        current = self.gens[self.currentGen].flatten()
        filtered = self.filterApply(self.currentGen)
        nextGen = []
        for index,element in enumerate(filtered.flatten()):
            nextGen.append(self.rules(element,current[index]))
        self.currentGen+=1
        self.gens.append(np.array(nextGen).reshape(self.size, self.size))
        
    def showAsImage(self,gen):
        img = Image.fromarray(((1-self.gens[gen]) * 255).astype('uint8'), mode='L')
        plt.figsize=(30, 30)
        im = plt.imshow(img,cmap='hot')

### Example creation of an 5x5 game.

In [ ]:
game = gameCreator(5)
game.grid

### We can get the next generation with obj.nextGen()

In [ ]:
game.nextGen()
game.gens

### Seeing as image

In [ ]:
game.showAsImage(1) #Pass the desired generation

### With gameFactory we can creat alot of games fast, keeping only the first and last;

In [ ]:
b = gameFactory(100) #100 games
b.generate() #size 25x25 default 

### Here we use the to_df() method, to convert our data generated and create as pandas dataframe.

In [ ]:
df = b.to_df()
df.delta.value_counts(normalize=True) #Here we can see that our delta is distributed

In [ ]:
df.head(5)

### And finally, lets create nth games.


In [ ]:
df = None

numberGames = 100 ##You may change here.

ini = time.time()
b = gameFactory(numberGames)
b.generate()
end = time.time()
print("Time required to generate the games : {}".format(end-ini))
ini = time.time()
df = b.to_df()
end = time.time()
print("Time required to generate the dataframe : {}".format(end-ini))

### And here we have nth samples created in df.

In [ ]:
df.head(10)

##     **Estimated time for creation:**

*  100 samples : 8 seconds
*  1000 samples : 20 seconds
*  10000 samples : 140 seconds